# Time dependent D3D simulation

In [ ]:
using Plots;
using FUSE
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
@time begin
#ini, act = FUSE.case_parameters(:D3D, 180893); #NBI with balanced torque
#ini, act = FUSE.case_parameters(:D3D, 133221); # ECH
#ini, act = FUSE.case_parameters(:D3D, 200000); # ECH
#ini, act = FUSE.case_parameters(:D3D, 170325); # ECH 1.67924
ini, act = FUSE.case_parameters(:D3D, 168830);
#@time ini, act = FUSE.case_parameters(:D3D, 168830);
#ini, act = FUSE.case_parameters(:D3D, 200204); # beam power scan, rotation, lower_to_upper null
#ini, act = FUSE.case_parameters(:D3D, 200021; EFIT_tree="EFIT01"); # negative B
#ini, act = FUSE.case_parameters(:D3D, 173389);
end
@checkin :fetch ini act

In [ ]:
@checkout :fetch ini act;
ini.time.simulation_start = ini.general.dd.equilibrium.time_slice[2].time
act.ActorReplay.replay_dd = ini.general.dd
dd =IMAS.dd()
@time FUSE.init!(dd,ini,act);
@checkin :init dd ini act;

In [ ]:
@checkout :init dd ini act;
experiment_LH = FUSE.LH_analysis(dd, 1.8, 2.2, 2.1; do_plot=true);
#experiment_LH = FUSE.LH_analysis(dd, 1.2, 1.5; do_plot=true);

If the simulation is starting from experimental conditions (ie. we have not changed the equilibrium, profiles, sources when the time dependent simulation starts), our plasma should already be in a self-consistent state and we are ready to start our time dependent simulation. If the simulation does not start from know experimental conditions, we would need to call the `ActorStationaryPlasma` to start from a know consistent state.

In [ ]:
@checkout :init dd act;
act.ActorPedestal.model = :dynamic
act.ActorPedestal.tau_n = experiment_LH.tau_n
act.ActorPedestal.tau_t = experiment_LH.tau_t
act.ActorWPED.ped_to_core_fraction = experiment_LH.W_ped_to_core_fraction
act.ActorEPED.ped_factor = 0.7
act.ActorPedestal.T_ratio_pedestal = 1.0 # Ti/Te in the pedestal
if true
    # density and Zeff from experiment
    act.ActorPedestal.density_ratio_L_over_H = 1.0
    act.ActorPedestal.zeff_ratio_L_over_H = 1.0
else
    # density can go from L to H mode at a different time
    act.ActorPedestal.density_ratio_L_over_H = experiment_LH.ne_L_over_H
    act.ActorPedestal.zeff_ratio_L_over_H = experiment_LH.zeff_L_over_H
    dd.pulse_schedule.density_control.n_e_line.reference = experiment_LH.ne_H
    dd.pulse_schedule.density_control.zeff_pedestal.reference = experiment_LH.zeff_H
end

if false
    # LH-transition from LH scaling law
    act.ActorPedestal.mode_transitions = missing
else
    # LH-transition at user-defined times
    act.ActorPedestal.mode_transitions = experiment_LH.mode_transitions
    act.ActorPedestal.mode_transitions[5.2] = :L_mode
    @show act.ActorPedestal.mode_transitions
end

act.ActorEquilibrium.model = :FRESCO # EGGO

act.ActorFRESCO.nR = 65
act.ActorFRESCO.nZ = 65

act.ActorNeutralFueling.τp_over_τe = 0.5

act.ActorFluxMatcher.evolve_plasma_sources = false
act.ActorFluxMatcher.max_iterations = 10
act.ActorFluxMatcher.verbose = false
act.ActorFluxMatcher.evolve_pedestal = false

act.ActorTGLF.tglfnn_model = "sat1_em_d3d"
#act.ActorTGLF.model = :GKNN
#act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"

dd.global_time = ini.general.dd.equilibrium.time_slice[2].time # start_time
δt = 0.025
final_time = 5.7
act.ActorDynamicPlasma.Nt = Int(ceil((final_time - dd.global_time) / δt))
act.ActorDynamicPlasma.Δt = final_time - dd.global_time

act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = true
act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = true

act.ActorDynamicPlasma.ip_controller = false
act.ActorDynamicPlasma.time_derivatives_sources = true

# act.ActorCurrent.model = :replay
# act.ActorEquilibrium.model = :replay
# act.ActorCoreTransport.model = :replay
# act.ActorPedestal.model = :replay
# act.ActorHCD.ec_model = :replay
# act.ActorHCD.ic_model = :replay
# act.ActorHCD.lh_model = :replay
# act.ActorHCD.nb_model = :replay
# act.ActorHCD.pellet_model = :replay
# act.ActorHCD.neutral_model = :none

@time actor = FUSE.ActorDynamicPlasma(dd, act; verbose=true);
@checkin :analysis dd ini act actor;

In [ ]:
@checkin :time_dep dd ini act actor;

In [ ]:
@checkout :time_dep dd act
using Interact
dd1 = FUSE.checkpoint[:init].dd
IMAS.trim_time!(dd, (-Inf, dd.global_time-1E-6))
@manipulate for time0 in slider(dd.equilibrium.time, value=dd.global_time, label="time")
    # p=plot(dd1.equilibrium; time0, color=:black)
    # plot!(dd.equilibrium; time0)
    # plot!(p[1], dd.wall)
    
    FUSE.plot_plasma_overview(dd, time0; dd1=dd1, size=(1400,1000), aggregate_hcd=true)#, min_power=1E4)

#     # plot(dd.core_transport; time0)

#     # plot(dd.core_sources; time0, aggregate_radiation=true, aggregate_hcd=true)
    
#     IMAS.ylim(Dict{Int,Float64}(
#         3 => 3.5, 4 => 1E20,
#         #-5 => -0.25, 5 => 1.5,
#         -6 => -0.15, 6 => 0.5, -7 => -0.15, 7 => 0.5, -8 => -1E20, 8 => 1.E20,
#         -10 => 0.0, 10 => 0.101, -11 => 0.0, 11 => 0.101, -12 => -2.0E19, 12 => 2.0E19))
end

Now we can plot the results, either as a animated gif

In [ ]:
@checkout :time_dep dd;

dd1 = FUSE.checkpoint[:init].dd
times = dd.core_profiles.time[1:end]
prog = FUSE.ProgressMeter.Progress(length(times))
a = @animate for (k, time0) in enumerate(times)
    FUSE.ProgressMeter.next!(prog; showvalues=(("k", k), ("time", time0)))
    # if k < 114
    #     continue
    # end

    p=plot(dd1.equilibrium; time0, color=:black)
    plot!(dd.equilibrium; time0)
    plot!(p[1],dd.wall)
    
#     FUSE.plot_plasma_overview(dd, time0; dd1, aggregate_hcd=true)

#     IMAS.ylim(Dict{Int,Float64}(
#         3 => 3.5, 4 => 1E20,
#         #-5 => -0.25, 5 => 1.5,
#         -6 => -0.15, 6 => 0.5, -7 => -0.15, 7 => 0.5, -8 => -1E20, 8 => 1.E20,
#         -10 => 0.0, 10 => 0.101, -11 => 0.0, 11 => 0.101, -12 => -2.0E19, 12 => 2.0E19))

    #savefig("D3D_168830_sawteeth_control/D3D_time_dep__$k.png")
end